# Berea field creation

You need the original dataset from Tidwell et al (1998) and copy the each face individually to the clipboard.

In [1]:
import pandas as pd
import numpy as np

In [4]:
def paste_from_clipboard():
    # read the data
    df = pd.read_clipboard(skiprows=3, decimal=',', header=None, usecols=[0,1,2,7,8,9,10])
    valnames = ['cm15', 'cm31', 'cm63', 'cm127']
    df.columns = ['x', 'y', 'z', *valnames]

    fields = {name: _fill_field(df, name) for name in valnames}

    return fields
    
def _fill_field(df: pd.DataFrame, colname: str):
    if colname not in df.columns:
        raise AttributeError(f"Can't find column {colname}")

    # generate the coordinate mapping
    xidx = {c: i for i, c in enumerate(np.unique(df.x))}
    yidx = {c: i for i, c in enumerate(np.unique(df.y))}
    zidx = {c: i for i, c in enumerate(np.unique(df.z))}

    # build the empty field and fill
    if len(xidx) == 1:
        field = np.ones((len(yidx), len(zidx))) * np.nan
        for _, row in df.iterrows():
            field[yidx[row.y], zidx[row.z]] = row.loc[colname]
    
    elif len(yidx) == 1:
        field = np.ones((len(xidx), len(zidx))) * np.nan
        for _, row in df.iterrows():
            field[xidx[row.x], zidx[row.z]] = row.loc[colname]
    
    elif len(zidx) == 1:
        field = np.ones((len(xidx), len(yidx))) * np.nan
        for _, row in df.iterrows():
            field[xidx[row.x], yidx[row.y]] = row.loc[colname]
    
    else:
        raise AttributeError('The data is corrupted, could not identify used axis')

    return field

Before you run the next cell, you need to copy the face data and set a name below

In [18]:
fields = paste_from_clipboard()
print(f"Parsed {len(fields)} fields")

Parsed 4 fields


In [19]:
face_name = 'Berea Face 4 - %s.txt'

for name, field in fields.items():
    np.savetxt(face_name % name, field)